# Basic Q&A System

### Sentence Retrival

In [1]:
import math,numpy,json,re,nltk
import time,re,os.path,sys
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import defaultdict
from numpy import multiply
from math import sqrt
from nltk.tokenize import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os.path as path
from collections import OrderedDict
from nltk.stem.wordnet import WordNetLemmatizer as WNL
from sklearn.feature_extraction import DictVectorizer
from nltk import FreqDist, DictionaryProbDist
from operator import add

#### Load data from json file

In [2]:
t0 = time.time()
filename_ls = ['QA_train.json']
dataset = []
train_path = path.abspath('data/QA_train.json')

dataset = []
with open(train_path) as f:
    for line in f:
        dataset+=(json.loads(line))
print "Import Successful and there are totally",len(dataset),'documents in this dataset'

Import Successful and there are totally 360 documents in this dataset


#### Build modle and evaluate the result

In [3]:
stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)
stemmer = nltk.stem.PorterStemmer() 

def my_tokenizer(doc):
    terms = set()
    for token in nltk.word_tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
            terms.add(stemmer.stem(token.lower()))
    return list(terms)


class MostRelevantSentenceModel(object):
    def __init__(self, vectorizer, collection_matrix):
        self.vectorizer = vectorizer
        self.collection_matrix = collection_matrix
        feature_array = vectorizer.get_feature_names()
        self.features = dict()
        for index in range(len(feature_array)):
            term = feature_array[index]
            self.features[term] = index

    def predict(self, queies):
        predictions = [self.inverted_index_score(i) for i in  queies]
        return predictions

    def inverted_index_score(self, query_sent):
        """
        now we implement inverted index to handle query
        
        :param query_sent: 
        :return: 
        
        """
        query_words = my_tokenizer(query_sent)
        score = defaultdict(float)

        for w in query_words:
            try:
                col_i = self.features[w]
                inverted_ix = self.collection_matrix[:, col_i]
                for di in range(inverted_ix.shape[0]):
                    score[di] += inverted_ix[di, 0]
            except KeyError:
                pass

        index_score = sorted(score.items(), key=lambda (k, v): v, reverse=True)

        if index_score:
            top10_doc_index = [i[0] for i in index_score[:10]]
            return top10_doc_index
        else:
            print 'error occured' ,query_sent
            return -1, 0

In [4]:
def build_model_and_evaluate(model, query, answer_i ,report=False):
    evaluate_row = []
    pred = model.predict(query)
    # score
    ac_index = 0
    quest_index = 0
    for pred_index, actual_index in zip(pred,answer_i):
        drow = dict()
        if report:
            print pred_index, '\t', actual_index
        drow['question_i'] = quest_index
        drow['prediction_i'] = pred_index
        drow['actual_yi'] = actual_index
        quest_index += 1
        if pred_index[0] == actual_index:
            drow['correctness'] = 1
            ac_index += 1
        else:
            drow['correctness'] = None
        evaluate_row.append(drow)
    accuracy = ac_index / len(document)
    #print('accuracy: ', accuracy)
    return evaluate_row, accuracy


#build model for each document collaction

for document in dataset:
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1, use_idf=True,norm='l1',stop_words=None, tokenizer=my_tokenizer)
    document_collections_sents = document['sentences']
    tfidf_matrix = tfidf_vectorizer.fit_transform(document_collections_sents)
    document['model'] = MostRelevantSentenceModel(vectorizer=tfidf_vectorizer,collection_matrix=tfidf_matrix)

#### Output with predtion and actual values

In [5]:
model1 = dataset[3]['model']
questions = [i['question'] for i in dataset[3]['qa']]
query = questions[24]
answer_i = [i['answer_sentence'] for i in dataset[3]['qa']]

#build_model_and_evaluate(model1, questions, answer_i)

In [6]:
import csv
csv_file = open('evaluatin_result.csv', mode='w',)
fieldnames = ['document_i', 'question_i', 'prediction_i','actual_yi', 'correctness', 'question', 'prediction_sentence', 'actual_y_sentence']
writer = csv.DictWriter(csv_file, fieldnames=fieldnames, )
writer.writeheader()

ddi = 0
for document in dataset:
    questions = [i['question'] for i in document['qa']]
    answer_indexs = [i['answer_sentence'] for i in document['qa']]
    model = document['model']
    table, acc = build_model_and_evaluate(model, questions, answer_indexs)
    doc_sents = document['sentences']
    for t in table:
        t['document_i'] = ddi
        t['question'] = questions[t['question_i']].encode('utf-8')
        t['prediction_sentence'] = doc_sents[t['prediction_i'][0]].encode('utf-8')
        t['actual_y_sentence'] =doc_sents[t['actual_yi']].encode('utf-8')
        writer.writerow(t)

    ddi += 1
print 'Running time count:', time.time() - t0

error occured Where were they?
error occured Where are most of them from?
error occured When was JFK shot?
error occured What is Renaixenca?
error occured What was his assigned punishment ?
error occured Why is this a bad practice?
error occured What happened in the Glided Age/
error occured What was the cause of this?
error occured Who were the rioters?
error occured k
error occured j
error occured n
error occured b
error occured v
error occured What are riwaaydo?
error occured When did Ruik past away?
error occured When did the USSR dissolve?
error occured Why did this occur?
error occured What does ROM stand for?
error occured dd
error occured dd
error occured dd
error occured dd
error occured d
error occured Where was Apprian from?
error occured What did the strikers achieve?
error occured What does the communicator do?
Running time count: 1494.68600011


In [7]:
csv_file = 'evaluatin_result.csv'
with open(csv_file) as csvfile:
    readCSV = csv.DictReader(csvfile, delimiter=',')
    for row in readCSV:
        print(row['prediction_i'])

[149, 391, 150, 253, 242, 89, 191, 325, 103, 188]
[310, 387, 89, 172, 13, 388, 21, 271, 392, 323]
[441, 271, 432, 450, 109, 29, 19, 209, 160, 260]
[1, 137, 429, 148, 55, 316, 89, 421, 329, 459]
[152, 217, 89, 161, 267, 54, 18, 234, 122, 228]
[21, 387, 3, 89, 429, 13, 158, 299, 301, 294]
[5, 461, 25, 359, 131, 148, 316, 243, 421, 459]
[148, 316, 202, 6, 421, 459, 284, 204, 137, 271]
[387, 183, 89, 7, 458, 13, 461, 8, 381, 302]
[243, 387, 237, 89, 25, 13, 388, 157, 21, 448]
[14, 402, 454, 9, 15, 289, 239, 224, 320, 288]
[10, 209, 11, 421, 25, 359, 14, 391, 143, 335]
[209, 12, 14, 359, 25, 391, 61, 157, 461, 110]
[11, 39, 206, 71, 151, 9, 0, 1, 2, 3]
[206, 359, 11, 143, 461, 110, 39, 25, 71, 151]
[13, 21, 15, 387, 199, 25, 23, 206, 103, 388]
[14, 15, 387, 13, 364, 388, 89, 21, 397, 25]
[13, 21, 103, 404, 206, 398, 249, 199, 292, 0]
[209, 310, 286, 14, 35, 25, 66, 243, 381, 391]
[221, 407, 217, 368, 367, 429, 141, 430, 289, 293]
[34, 58, 32, 161, 98, 30, 56, 26, 271, 219]
[208, 407, 217, 3

### Entity Extraction

In [8]:
def json_load_byteified(file_handle):
    return _byteify(json.load(file_handle, object_hook=_byteify),ignore_dicts=True)

def _byteify(data, ignore_dicts = False):
    # if this is a unicode string, return its string representation
    if isinstance(data, unicode):
        return data.encode('utf-8')
    # if this is a list of values, return list of byteified values
    if isinstance(data, list):
        return [ _byteify(item, ignore_dicts=True) for item in data ]
    # if this is a dictionary, return dictionary of byteified keys and values
    # but only if we haven't already byteified it
    if isinstance(data, dict) and not ignore_dicts:
        return {
            _byteify(key, ignore_dicts=True): _byteify(value, ignore_dicts=True)
            for key, value in data.iteritems()
        }
    # if it's anything else, return it in its original form
    return data

with open("data/QA_train.json") as json_file:
    json_data = json_load_byteified(json_file)
print 'import success'

import success


In [10]:
cwd = os.getcwd()
st = StanfordNERTagger(cwd+'\data\english.all.3class.distsim.crf.ser.gz',cwd+'\data\stanford-ner.jar')

if not os.path.isfile("NER.json"):    
    start = time.time()
    progressT = len(json_data)    
    listOfDocument=[]
    i=0
    for jd in json_data:
        listOfDocument.extend([st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x))) for x in jd['sentences']])])
        i+=1
        sys.stdout.write('\r')
        sys.stdout.write("%d%%" % (i*100/progressT))
        sys.stdout.flush()
    for document in range(0,len(listOfDocument)):
        for sentence in range(0,len(listOfDocument[document])):
            for word in range(0,len(listOfDocument[document][sentence])):   
                listOfDocument[document][sentence][word]= (listOfDocument[document][sentence][word][0],listOfDocument[document][sentence][word][1] if not listOfDocument[document][sentence][word][0].isdigit() else u'NUMBER')
    with open('NER.json', 'w') as outfile:
        json.dump(listOfDocument, outfile)
    end = time.time()
    print '\nTime spending:',end - start    
else:    
    print 'there is a file'
with open("NER.json") as json_file:
        json_dataNER = json_load_byteified(json_file)

LookupError: 

===========================================================================
NLTK was unable to find the java file!
Use software specific configuration paramaters or set the JAVAHOME environment variable.
===========================================================================